In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [9]:
df_zlut=pd.read_excel('z_table.xlsx')
df_zlut

,Time(s),Velocity (mm/s),Degrees,Displacement(mm)
0,0.06,26.473521,-38.4,19.467589
1,0.12,25.556870,-36.8,17.934177
2,0.18,24.620291,-35.2,16.456959
3,0.24,23.664514,-33.6,15.037088
4,0.30,22.690283,-32.0,13.675671
5,0.36,21.698360,-30.4,12.373770
6,0.42,20.689516,-28.8,11.132399
7,0.48,19.664540,-27.2,9.952526
8,0.54,18.624230,-25.6,8.835073
9,0.60,17.569397,-24.0,7.780909


In [ ]:
pvt = device.get_pvt(1)

pvt_buffer = device.get_pvt_buffer(1)
pvt_buffer.erase()

# set up PVT to store points to PVT buffer 1 and
# to use the first axis for unit conversion
pvt.setup_store(pvt_buffer, 1, 1)

# add PVT points from LUT in Excel file
for index, row in df.iterrows():
    # Add each point in a loop
    pvt.point([
        Measurement(row["Displacement(mm)"], Units.LENGTH_MILLIMETRES), # x
    ], [
        Measurement(row["Velocity (mm/s)"], Units.VELOCITY_MILLIMETRES_PER_SECOND), # v_x
    ], Measurement(point["Time(s)"], Units.TIME_SECONDS)) # time

# finish writing to the buffer
pvt.disable()


In [ ]:
# execute:
pvt.setup_live(3) # Execute on axis 3

# the sequence will now execute the stored points
pvt.call(pvt_buffer)